In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Configure plot styles
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

def analyze_missing_values_by_time(df, patient_id_col, time_step_col):
    """
    Analyze missing values in the dataset, focusing on their distribution across time.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataset with patient measurements over time
    patient_id_col : str
        Column name for patient ID
    time_step_col : str
        Column name for time step
        
    Returns:
    --------
    missing_summary : pandas DataFrame
        Summary of missing values by feature and time
    """
    # Identify feature columns (exclude metadata columns)
    non_feature_cols = [patient_id_col, time_step_col, 'icu_mortality']
    feature_cols = [col for col in df.columns if col not in non_feature_cols]
    
    # Calculate overall missing percentage
    missing_overall = df[feature_cols].isna().mean().sort_values(ascending=False)
    print(f"Overall dataset has {missing_overall.mean()*100:.2f}% missing values across all features")
    
    # Create time bins for visualization (convert minutes to hours)
    if df[time_step_col].max() > 600:  # If time is in minutes
        df['time_hour'] = df[time_step_col] / 60
        time_bins = [0, 4, 8, 12, 16, 20, 24]
        df['time_bin'] = pd.cut(df['time_hour'], bins=time_bins, labels=[f"{i}-{i+4}" for i in range(0, 24, 4)])
    else:
        df['time_hour'] = df[time_step_col]  # Assume already in hours
        time_bins = list(range(0, df['time_hour'].max() + 4, 4))
        df['time_bin'] = pd.cut(df['time_hour'], bins=time_bins)
    
    # Calculate missing values by time bin
    missing_by_time = []
    for time_bin, group in df.groupby('time_bin'):
        bin_missing = group[feature_cols].isna().mean()
        bin_missing['time_bin'] = time_bin
        bin_missing['sample_count'] = len(group)
        missing_by_time.append(bin_missing)
    
    missing_by_time_df = pd.DataFrame(missing_by_time).set_index('time_bin')
    
    # Plot missing value patterns
    plt.figure(figsize=(16, 12))
    
    # 1. Top features with missing values
    plt.subplot(2, 2, 1)
    top_missing = missing_overall.head(15)
    sns.barplot(x=top_missing.values * 100, y=top_missing.index)
    plt.title('Top 15 Features with Missing Values')
    plt.xlabel('Missing Percentage (%)')
    plt.ylabel('Feature')
    
    # 2. Missing values by time bin
    plt.subplot(2, 2, 2)
    # Calculate average missing rate for each time bin
    avg_missing = missing_by_time_df[feature_cols].mean(axis=1) * 100
    sample_counts = missing_by_time_df['sample_count']
    
    ax1 = plt.gca()
    ax1.bar(avg_missing.index, avg_missing.values, color='skyblue', alpha=0.7)
    ax1.set_xlabel('Time Bin (hours)')
    ax1.set_ylabel('Average Missing Value (%)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    
    # Add sample count on secondary axis
    ax2 = ax1.twinx()
    ax2.plot(sample_counts.index, sample_counts.values, 'r-', marker='o')
    ax2.set_ylabel('Sample Count', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    plt.title('Average Missing Values by Time')
    
    # 3. Heatmap of missing values for top features by time
    plt.subplot(2, 1, 2)
    top_missing_features = missing_overall.head(10).index
    heatmap_data = missing_by_time_df[top_missing_features] * 100
    sns.heatmap(heatmap_data.T, annot=True, fmt='.1f', cmap='YlOrRd')
    plt.title('Missing Value Percentage by Time for Top 10 Features')
    plt.xlabel('Time Bin (hours)')
    plt.ylabel('Feature')
    
    plt.tight_layout()
    
    # Calculate missingness patterns
    missing_patterns = df[feature_cols].isna().sum(axis=1)
    pattern_counts = missing_patterns.value_counts()
    
    # Create summary of missingness
    missing_summary = pd.DataFrame({
        'feature': missing_overall.index,
        'missing_pct': missing_overall.values * 100,
        'complete_count': df.shape[0] - df[missing_overall.index].isna().sum(),
        'missing_count': df[missing_overall.index].isna().sum()
    })
    
    # Print missingness summary
    print(f"\nMissing value analysis complete")
    print(f"Found {len(pattern_counts)} distinct missingness patterns")
    print(f"Most common pattern has {pattern_counts.iloc[0]} occurrences ({pattern_counts.iloc[0]/len(df)*100:.1f}% of records)")
    
    return missing_summary

def identify_measurement_patterns(df, patient_id_col, time_step_col):
    """
    Identify patterns in when measurements are taken for different patients.
    This helps understand if some features are measured on a different schedule.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataset with patient measurements over time
    patient_id_col : str
        Column name for patient ID
    time_step_col : str
        Column name for time step
    """
    # Identify feature columns (exclude metadata columns)
    non_feature_cols = [patient_id_col, time_step_col, 'icu_mortality']
    feature_cols = [col for col in df.columns if col not in non_feature_cols]
    
    # Select a subset of patients for analysis
    sample_patients = np.random.choice(df[patient_id_col].unique(), size=min(5, df[patient_id_col].nunique()), replace=False)
    
    # Create visualization
    plt.figure(figsize=(15, 10))
    
    for i, patient_id in enumerate(sample_patients):
        patient_data = df[df[patient_id_col] == patient_id].sort_values(time_step_col)
        
        # Convert time to hours for plotting
        time_hours = patient_data[time_step_col] / 60 if patient_data[time_step_col].max() > 100 else patient_data[time_step_col]
        
        # Create a presence map (0 = missing, 1 = present)
        presence_map = (~patient_data[feature_cols].isna()).astype(int)
        
        # Plot the presence map
        plt.subplot(len(sample_patients), 1, i+1)
        sns.heatmap(presence_map.T, cmap='Blues', cbar=False)
        plt.yticks(np.arange(len(feature_cols))+0.5, feature_cols, fontsize=8)
        plt.xticks(np.arange(0, len(patient_data), max(1, len(patient_data)//10)), 
                   [f"{h:.1f}" for h in time_hours[::max(1, len(patient_data)//10)]], 
                   rotation=45)
        plt.title(f"Patient {patient_id} - Measurement Presence (0=missing, 1=present)")
        plt.xlabel("Time (hours)")
    
    plt.tight_layout()
    plt.suptitle("Feature Measurement Patterns by Patient", fontsize=16)
    plt.subplots_adjust(top=0.95)

def impute_missing_values_intelligently(df, patient_id_col, time_step_col, missing_summary):
    """
    Impute missing values using strategies appropriate to medical time series data.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataset with patient measurements over time
    patient_id_col : str
        Column name for patient ID
    time_step_col : str
        Column name for time step
    missing_summary : pandas DataFrame
        Summary of missing values by feature
        
    Returns:
    --------
    df_imputed : pandas DataFrame
        Dataset with imputed values
    imputation_stats : dict
        Statistics about the imputation process
    """
    print("Starting intelligent imputation process...")
    df_imputed = df.copy()
    
    # Identify feature columns (exclude metadata columns)
    non_feature_cols = [patient_id_col, time_step_col, 'icu_mortality', 'time_hour', 'time_bin']
    feature_cols = [col for col in df.columns if col not in non_feature_cols]
    
    # Track imputation statistics
    imputation_stats = {
        'forward_fill_count': 0,
        'backward_fill_count': 0,
        'median_fill_count': 0,
        'total_missing': df[feature_cols].isna().sum().sum()
    }
    
    # Group by patient to maintain patient boundaries
    grouped = df_imputed.groupby(patient_id_col)
    
    # 1. Forward fill within each patient (carry last observation forward)
    print("Applying forward fill within each patient...")
    before_ffill = df_imputed[feature_cols].isna().sum().sum()
    df_imputed[feature_cols] = grouped[feature_cols].ffill()
    after_ffill = df_imputed[feature_cols].isna().sum().sum()
    imputation_stats['forward_fill_count'] = before_ffill - after_ffill
    
    # 2. Backward fill for initial values
    print("Applying backward fill for initial values...")
    before_bfill = df_imputed[feature_cols].isna().sum().sum()
    df_imputed[feature_cols] = grouped[feature_cols].bfill()
    after_bfill = df_imputed[feature_cols].isna().sum().sum()
    imputation_stats['backward_fill_count'] = before_bfill - after_bfill
    
    # 3. For remaining NaNs, use feature-specific strategies
    remaining_missing = df_imputed[feature_cols].isna().sum()
    
    if remaining_missing.sum() > 0:
        print(f"Handling remaining {remaining_missing.sum()} missing values...")
        
        for feature in feature_cols:
            if df_imputed[feature].isna().sum() > 0:
                # Use median imputation for remaining values
                feature_median = df_imputed[feature].median()
                missing_count = df_imputed[feature].isna().sum()
                df_imputed[feature] = df_imputed[feature].fillna(feature_median)
                imputation_stats['median_fill_count'] += missing_count
                
                print(f"  - {feature}: {missing_count} values imputed with median {feature_median:.3f}")
    
    # Final check
    final_missing = df_imputed[feature_cols].isna().sum().sum()
    
    imputation_stats['percent_forward_filled'] = imputation_stats['forward_fill_count'] / imputation_stats['total_missing'] * 100
    imputation_stats['percent_backward_filled'] = imputation_stats['backward_fill_count'] / imputation_stats['total_missing'] * 100
    imputation_stats['percent_median_filled'] = imputation_stats['median_fill_count'] / imputation_stats['total_missing'] * 100
    
    print("\nImputation complete:")
    print(f"- Total missing values: {imputation_stats['total_missing']}")
    print(f"- Forward filled: {imputation_stats['forward_fill_count']} ({imputation_stats['percent_forward_filled']:.1f}%)")
    print(f"- Backward filled: {imputation_stats['backward_fill_count']} ({imputation_stats['percent_backward_filled']:.1f}%)")
    print(f"- Median filled: {imputation_stats['median_fill_count']} ({imputation_stats['percent_median_filled']:.1f}%)")
    print(f"- Remaining missing: {final_missing}")
    
    return df_imputed, imputation_stats

def validate_imputation(df_original, df_imputed, patient_id_col, time_step_col):
    """
    Validate the imputation by comparing original vs. imputed data
    
    Parameters:
    -----------
    df_original : pandas DataFrame
        Original dataset with missing values
    df_imputed : pandas DataFrame
        Dataset with imputed values
    patient_id_col : str
        Column name for patient ID
    time_step_col : str
        Column name for time step
    """
    # Identify feature columns
    non_feature_cols = [patient_id_col, time_step_col, 'icu_mortality', 'time_hour', 'time_bin']
    feature_cols = [col for col in df_original.columns if col not in non_feature_cols]
    
    # Calculate overall imputation rate
    imputed_cells = ((df_original[feature_cols].isna()) & (~df_imputed[feature_cols].isna())).sum().sum()
    total_cells = df_original.shape[0] * len(feature_cols)
    imputation_rate = imputed_cells / total_cells * 100
    
    print(f"Imputation validation:")
    print(f"- Total feature cells: {total_cells}")
    print(f"- Imputed cells: {imputed_cells} ({imputation_rate:.2f}%)")
    
    # Visualize imputation for a few patients
    sample_patients = np.random.choice(df_original[patient_id_col].unique(), 
                                      size=min(3, df_original[patient_id_col].nunique()), 
                                      replace=False)
    
    for patient_id in sample_patients:
        plt.figure(figsize=(15, 12))
        
        # Get patient data from original and imputed datasets
        patient_orig = df_original[df_original[patient_id_col] == patient_id].sort_values(time_step_col)
        patient_imputed = df_imputed[df_imputed[patient_id_col] == patient_id].sort_values(time_step_col)
        
        # Convert time to hours for plotting
        time_hours = patient_orig[time_step_col] / 60 if patient_orig[time_step_col].max() > 100 else patient_orig[time_step_col]
        
        # Select a few interesting features to plot
        features_to_plot = []
        
        # Add features with high missingness first
        high_missing_features = df_original[feature_cols].isna().mean().sort_values(ascending=False).head(6).index
        features_to_plot.extend(high_missing_features)
        
        # Add some features with low missingness
        low_missing_features = df_original[feature_cols].isna().mean().sort_values().head(2).index
        features_to_plot.extend(low_missing_features)
        
        # Limit to 8 features
        features_to_plot = features_to_plot[:8]
        
        # Plot each feature
        for i, feature in enumerate(features_to_plot):
            ax = plt.subplot(4, 2, i+1)
            
            # Original values (non-NaN)
            valid_mask = ~patient_orig[feature].isna()
            plt.plot(time_hours[valid_mask], patient_orig[feature][valid_mask], 'o-', color='blue', label='Original')
            
            # Imputed values (NaN in original, non-NaN in imputed)
            imputed_mask = (patient_orig[feature].isna()) & (~patient_imputed[feature].isna())
            plt.plot(time_hours[imputed_mask], patient_imputed[feature][imputed_mask], 'x', color='red', markersize=8, label='Imputed')
            
            # Connect all points in imputed dataset
            plt.plot(time_hours, patient_imputed[feature], '--', color='gray', alpha=0.5, label='_nolegend_')
            
            plt.title(f"{feature}")
            plt.xlabel("Time (hours)")
            
            # Only show legend on first plot
            if i == 0:
                plt.legend()
        
        plt.suptitle(f"Patient {patient_id} - Original vs. Imputed Values", fontsize=16)
        plt.tight_layout()
        plt.subplots_adjust(top=0.92)

# Usage example with mimic_expanded_metrics_24hr.csv:

def analyze_missing_data_pipeline(csv_path="mimic_expanded_metrics_24hr.csv"):
    """
    Complete pipeline for analyzing and imputing missing data in the MIMIC dataset.
    
    Parameters:
    -----------
    csv_path : str
        Path to the CSV file (default: "mimic_expanded_metrics_24hr.csv")
    
    Returns:
    --------
    df_imputed : pandas DataFrame
        Dataset with imputed values
    missing_summary : pandas DataFrame
        Summary of missing values by feature
    imputation_stats : dict
        Statistics about the imputation process
    """
    print(f"Loading data from {csv_path}...")
    
    try:
        # Load the data
        df = pd.read_csv(csv_path)
        print(f"Data loaded successfully. Shape: {df.shape}")
        
        # Constants for column names
        PATIENT_ID_COLUMN = 'patient_id'
        TIME_STEP_COLUMN = 'time_step'
        TARGET_COLUMN = 'icu_mortality'
        
        # Check if key columns exist
        if PATIENT_ID_COLUMN not in df.columns:
            raise ValueError(f"'{PATIENT_ID_COLUMN}' column not found in the dataset")
            
        if TARGET_COLUMN not in df.columns:
            raise ValueError(f"'{TARGET_COLUMN}' column not found in the dataset")
            
        # Basic dataset statistics
        print(f"Number of unique patients: {df[PATIENT_ID_COLUMN].nunique()}")
        print(f"Target distribution: {df[TARGET_COLUMN].value_counts(normalize=True)}")
        
        # Extract time steps if needed
        if TIME_STEP_COLUMN not in df.columns:
            # Try to infer time steps from column names
            print("Time step column not found, attempting to infer from feature columns...")
            time_points = set()
            pattern = re.compile(r'(.+)_(\d+)min
)
            
            for col in df.columns:
                if col == PATIENT_ID_COLUMN or col == TARGET_COLUMN:
                    continue
                    
                match = pattern.match(col)
                if match:
                    time_point = int(match.groups()[1])
                    time_points.add(time_point)
            
            if time_points:
                print(f"Detected time points: {sorted(time_points)}")
                # Create a new dataframe with time step structure
                records = []
                for patient_id, patient_data in df.groupby(PATIENT_ID_COLUMN):
                    for time_point in sorted(time_points):
                        records.append({
                            PATIENT_ID_COLUMN: patient_id,
                            TIME_STEP_COLUMN: time_point,
                            TARGET_COLUMN: patient_data[TARGET_COLUMN].iloc[0]
                        })
                
                df_time = pd.DataFrame(records)
                print(f"Created time step structure with shape: {df_time.shape}")
                df = df_time
            else:
                TIME_STEP_COLUMN = 'timestamp'
                print("Couldn't infer time steps. Setting a placeholder time column.")
                df[TIME_STEP_COLUMN] = 0
        
        # 1. Analyze missing values
        print("\n=== Analyzing Missing Values ===")
        missing_summary = analyze_missing_values_by_time(df, PATIENT_ID_COLUMN, TIME_STEP_COLUMN)
        
        # 2. Identify measurement patterns
        print("\n=== Identifying Measurement Patterns ===")
        identify_measurement_patterns(df, PATIENT_ID_COLUMN, TIME_STEP_COLUMN)
        
        # 3. Perform intelligent imputation
        print("\n=== Performing Intelligent Imputation ===")
        df_imputed, imputation_stats = impute_missing_values_intelligently(df, PATIENT_ID_COLUMN, TIME_STEP_COLUMN, missing_summary)
        
        # 4. Validate imputation
        print("\n=== Validating Imputation ===")
        validate_imputation(df, df_imputed, PATIENT_ID_COLUMN, TIME_STEP_COLUMN)
        
        print("\nMissing data analysis and imputation pipeline complete!")
        
        # Save imputed data
        output_file = "mimic_expanded_metrics_24hr_imputed.csv"
        df_imputed.to_csv(output_file, index=False)
        print(f"Imputed data saved to {output_file}")
        
        return df_imputed, missing_summary, imputation_stats
        
    except Exception as e:
        print(f"Error in analysis pipeline: {str(e)}")
        raise

# Run the pipeline
df_imputed, missing_summary, imputation_stats = analyze_missing_data_pipeline("mimic_expanded_metrics_24hr.csv")

SyntaxError: unterminated string literal (detected at line 383) (284594113.py, line 383)